In [0]:
from pyspark.sql import functions as F

VOLUME_PATH = "/Volumes/medalhao/default/landing/"

tabelas_para_ingerir = {
    "olist_customers_dataset.csv": "medalhao.bronze.ft_consumidores",
    "olist_geolocation_dataset.csv": "medalhao.bronze.ft_geolocalizacao",
    "olist_order_items_dataset.csv": "medalhao.bronze.ft_itens_pedidos",
    "olist_order_payments_dataset.csv": "medalhao.bronze.ft_pagamentos_pedidos",
    "olist_order_reviews_dataset.csv": "medalhao.bronze.ft_avaliacoes_pedidos",
    "olist_orders_dataset.csv": "medalhao.bronze.ft_pedidos",
    "olist_products_dataset.csv": "medalhao.bronze.ft_produtos",
    "olist_sellers_dataset.csv": "medalhao.bronze.ft_vendedores",
    "product_category_name_translation.csv": "medalhao.bronze.dm_categoria_produtos_traducao"
}

print("Iniciando ingestão da camada Bronze...")

for arquivo_csv, nome_tabela_bronze in tabelas_para_ingerir.items():
    try:
        # Caminho completo para o arquivo CSV no Volume
        file_path = f"{VOLUME_PATH}{arquivo_csv}"
        
        # Ler o arquivo (com cabeçalho e inferindo o schema)
        df_raw = spark.read.csv(file_path, header=True, inferSchema=True)
        
        # Adicionar a coluna de timestamp da ingestão 
        df_with_timestamp = df_raw.withColumn("ingestion_timestamp", F.current_timestamp())
        
        # Salvar como tabela Delta no schema 'medalhao.bronze'
        df_with_timestamp.write.mode("overwrite").saveAsTable(nome_tabela_bronze)
        
        print(f"Tabela {nome_tabela_bronze} ingerida com sucesso.")
        
    except Exception as e:
        print(f"Erro ao ingerir {nome_tabela_bronze}: {e}")

print("Ingestão dos 9 arquivos CSV concluída.")

In [0]:
import requests
import json
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

dbutils.widgets.text("data_inicio", "01-01-2017")
dbutils.widgets.text("data_fim", "12-31-2018")

data_inicio_formatada = dbutils.widgets.get("data_inicio")
data_fim_formatada = dbutils.widgets.get("data_fim")

url_api = (
    f"https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/"
    f"CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?"
    f"@dataInicial='{data_inicio_formatada}'&@dataFinalCotacao='{data_fim_formatada}'"
    "&$select=dataHoraCotacao,cotacaoCompra&$format=json"
)

print(f"Buscando dados da API: {url_api}")

try:
    response = requests.get(url_api)
    response.raise_for_status() 
    data = response.json()
    cotacoes = data.get('value', [])
    
    if cotacoes:
        schema = StructType([
            StructField("dataHoraCotacao", StringType(), True),
            StructField("cotacaoCompra", DoubleType(), True)
        ])
        
        df_cotacao = spark.createDataFrame(cotacoes, schema=schema)
        
        nome_tabela_api = "medalhao.bronze.dm_cotacao_dolar"
        df_cotacao.write.mode("overwrite").saveAsTable(nome_tabela_api)
        
        print(f"SUCESSO: Tabela {nome_tabela_api} criada com {len(cotacoes)} registros.")
    else:
        print("AVISO: Nenhum dado de cotação retornado pela API para o período selecionado.")

except Exception as e:
    print(f"ERRO ao buscar dados da API: {e}")